In [14]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_table
from dash.dependencies import Input, Output
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.express as px
# from dash_table import DataTable, FormatTemplate

from calc import *

In [15]:
def create_mortage_range(principal, term, rmax=0.05, rstep=0.0125):
    df = pd.DataFrame(data={'rate':np.arange(0,rmax+rstep,rstep)})
    df['monthly_payment'] = df.apply(lambda x: calc_loan_monthly_payment(principal, x['rate'], term), axis=1)
    df['total_payment'] = df.apply(lambda x: totala_payment(x['monthly_payment'], term), axis=1)
    print(df)
    return df


def create_value_plot(principal, term, x='rate', y='monthly_payment', title='Monthly Payment by Interest Rate'):
    df = create_mortage_range(principal, term)
    fig = px.line(df, x=x, y=y, title=title)
    return fig


In [16]:
f = create_value_plot(510000, 360)
f.show()

NameError: name 'totala_payment' is not defined